In [1]:
# sentiments looks like this:
# [['ausdauer', 'ausdauernd', 'ausdauernde', 0.4], ...]
import re
sentiments = {}

regex = re.compile('^(\w+)\|\s+(-?\d+\.\d+)$')

In [2]:
def parse_sentiment(line):
    line = line.lower()
    main_re = re.compile(r'^([a-zäöüß]+)', re.IGNORECASE)
    sentiment_re = re.compile(r'(-?\d+.\d+)')
    forms_re = re.compile(r'([a-zäöüß,]+)$', re.IGNORECASE)
    
    sentiment = float(sentiment_re.findall(line)[0])
    result = main_re.findall(line)
    sentiments[result[0]] = sentiment
    permutations = [f.split(',') for f in forms_re.findall(line)]
    if permutations:
        for p in permutations[0]:
            sentiments[p] = sentiment

In [3]:
parse_sentiment('Unglück|NN	-0.5004	Unglückes,Unglücks,Unglücke,Unglücken')

In [4]:
with open('SentiWS_v1.8c/SentiWS_v1.8c_Positive.txt') as file:
    for line in file:
        parse_sentiment(line)

with open('SentiWS_v1.8c/SentiWS_v1.8c_Negative.txt') as file:
    for line in file:
        parse_sentiment(line)

In [5]:
def get_sentiment(text):
    words = [t.lower() for t in text.replace('.', '').replace(',', '').replace(';', '').split()]
    sentiment = 0
    matches = 0
    for w in words:
        s = sentiments.get(w)
        if s:
            sentiment += s
            matches += 1
    if matches:
        return sentiment / matches
    else:
        return 0

In [6]:
get_sentiment("Lob")

0.4445

In [7]:
get_sentiment("belohnung")

0.0813

In [8]:
# here we go; split each text into pieces of 500 words and calculate the sentiments
def calculate_sentiment(file, step_size=500):
    with open(file) as f:
        content = f.read().split()
        start = 0
        results = []
        while start < len(content):
            words = content[start:(start+step_size)]
            results.append(get_sentiment(' '.join(words)))
            start += step_size
        
        return results

In [9]:
import glob
import json

for path in glob.glob('./dumps/texts/*'):
    filename = path.split('/')[-1]
    identifier = filename.split('.')[0]
    with open('./dumps/sentiment/{}.json'.format(identifier), 'w') as f:
        f.write(json.dumps(calculate_sentiment(path)))

In [10]:
get_sentiment("Die Wirtschaft steckt in der Krise.")

-0.3621

In [11]:
get_sentiment("Ein fürchterliches Unglück ist über uns gekommen.")

-0.41835